In [1]:
import numpy as np
import hashlib
import time

from sha3 import sha3_oop, sha3_functional, sha3_numba, sha3_cuda

def benchmark_and_verify(bit_length, input_bytes, num_iterations):

    # Python SHA3 Implementation (OOP)
    start_time = time.perf_counter()
    for _ in range(num_iterations):
        sha3_256 = sha3_oop(bit_length)
        sha3_256.update(input_bytes)
        my_hash = sha3_256.hexdigest()
    my_time = time.perf_counter() - start_time

    # Python SHA3 Implementation (Functional)
    start_time = time.perf_counter()
    for _ in range(num_iterations):
        func_hash = sha3_functional(bit_length, input_bytes).hex()
    func_time = time.perf_counter() - start_time

    # Warm up
    sha3_numba(bit_length, input_bytes)
    sha3_cuda(bit_length, input_bytes, 1)
    
    # Python SHA3 Implementation (Numba)
    start_time = time.perf_counter()
    for _ in range(num_iterations):
        numba_hash = sha3_numba(bit_length, input_bytes).tobytes().hex()
    numba_time = time.perf_counter() - start_time

    # Python SHA3 Implementation (CUDA)
    start_time = time.perf_counter()
    cuda_hashes = sha3_cuda(bit_length, input_bytes, num_iterations)
    cuda_time = time.perf_counter() - start_time

    # Verify that all cuda hashes are the same
    cuda_hash = cuda_hashes[0]
    for i in range(1, len(cuda_hashes)):
        assert np.array_equal(cuda_hash, cuda_hashes[i]), f"Mismatch: {cuda_hash} != {cuda_hashes[i]}"
    cuda_hash = cuda_hash.tobytes().hex()

    # HashLib SHA3 Implementation
    hashlib_func = hashlib.sha3_256 if bit_length == 256 else hashlib.sha3_512
    start_time = time.perf_counter()
    for _ in range(num_iterations):
        lib_hash = hashlib_func(input_bytes).hexdigest()
    lib_time = time.perf_counter() - start_time
    
    print(f"Input Length: {len(input_bytes)} bytes, Iterations: {num_iterations}")
    print(f"Python  SHA3-{bit_length} Time: {my_time:.4f} s, Hashes/s: {num_iterations/my_time:.1f}, Length: {len(my_hash)}, Hash: {my_hash}")
    print(f"Func.   SHA3-{bit_length} Time: {func_time:.4f} s, Hashes/s: {num_iterations/func_time:.1f}, Length: {len(func_hash)}, Hash: {func_hash}")
    print(f"Numba   SHA3-{bit_length} Time: {numba_time:.4f} s, Hashes/s: {num_iterations/numba_time:.1f}, Length: {len(numba_hash)}, Hash: {numba_hash}")
    print(f"CUDA    SHA3-{bit_length} Time: {cuda_time:.4f} s, Hashes/s: {num_iterations/cuda_time:.1f}, Length: {len(cuda_hash)}, Hash: {cuda_hash}")
    print(f"HashLib SHA3-{bit_length} Time: {lib_time:.4f} s, Hashes/s: {num_iterations/lib_time:.1f}, Length: {len(lib_hash)}, Hash: {lib_hash}")

    # Verify Outputs
    assert my_hash == lib_hash, f"Mismatch: {my_hash} != {lib_hash}"
    assert func_hash == lib_hash, f"Mismatch: {func_hash} != {lib_hash}"
    assert numba_hash == lib_hash, f"Mismatch: {numba_hash} != {lib_hash}"
    assert cuda_hash == lib_hash, f"Mismatch: {cuda_hash} != {lib_hash}"
    print("Verification: PASSED\n")


In [2]:
BIT_LENGTHS = [256, 512] # SHA3-256, SHA3-512
NUM_ITERATIONS = 1000 # Number of iterations to run

for bit_length in BIT_LENGTHS:

    # First, test empty input
    benchmark_and_verify(bit_length, bytes(), NUM_ITERATIONS)

    # Test 'Hello, World!' input
    benchmark_and_verify(bit_length, bytes('Hello, World!', 'utf-8'), NUM_ITERATIONS)

    # Test random 1000 bytes
    benchmark_and_verify(bit_length, np.random.bytes(1000), NUM_ITERATIONS)

/home/nicolas/.local/share/virtualenvs/sha-3-numba-N12ohUSG/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/nicolas/.local/share/virtualenvs/sha-3-numba-N12ohUSG/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 8 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Input Length: 0 bytes, Iterations: 1000
Python  SHA3-256 Time: 1.2458 s, Hashes/s: 802.7, Length: 64, Hash: a7ffc6f8bf1ed76651c14756a061d662f580ff4de43b49fa82d80a4b80f8434a
Func.   SHA3-256 Time: 1.2274 s, Hashes/s: 814.7, Length: 64, Hash: a7ffc6f8bf1ed76651c14756a061d662f580ff4de43b49fa82d80a4b80f8434a
Numba   SHA3-256 Time: 0.0020 s, Hashes/s: 504821.3, Length: 64, Hash: a7ffc6f8bf1ed76651c14756a061d662f580ff4de43b49fa82d80a4b80f8434a
CUDA    SHA3-256 Time: 0.0008 s, Hashes/s: 1257523.1, Length: 64, Hash: a7ffc6f8bf1ed76651c14756a061d662f580ff4de43b49fa82d80a4b80f8434a
HashLib SHA3-256 Time: 0.0006 s, Hashes/s: 1730975.3, Length: 64, Hash: a7ffc6f8bf1ed76651c14756a061d662f580ff4de43b49fa82d80a4b80f8434a
Verification: PASSED

Input Length: 13 bytes, Iterations: 1000
Python  SHA3-256 Time: 1.2157 s, Hashes/s: 822.5, Length: 64, Hash: 1af17a664e3fa8e419b8ba05c2a173169df76162a5a286e0c405b460d478f7ef
Func.   SHA3-256 Time: 1.2303 s, Hashes/s: 812.8, Length: 64, Hash: 1af17a664e3fa8e419b8